In [6]:

# from lab_utils_uni import plt_house_x, plt_contour_wgrad, plt_divergence, plt_gradients
import math
import copy
import numpy as np
import matplotlib.pyplot as plot
plot.style.use('./utils_plot/deeplearning.mplstyle')

In [7]:
def compute_cost(x, y, w, b): 
    """
    Computes the cost function for linear regression. 

    ndarray: N-dimensional array: https://numpy.org/doc/stable/reference/arrays.ndarray.html

    Args: 
        x (ndarray (m,)):       data, m examples
        y (ndarray, (m,)):      target values
        w, b (scalar):          model parameters
    
    Returns 
        total_cost (float):     the cost of using w, b as the parameters for linear regression 
                                to fit the data points in x and y 
    """
    # number of the training examples
    m = x.shape[0]
    cost_sum = 0
    for i in range(m): 
        # given a training example, and parameters w, b, calculate prediction 
        # (prediction is the value of the function f_wb)
        f_wb = w * x[i] + b  # 𝑓𝑤,𝑏(𝑥(𝑖))=𝑤𝑥(𝑖)+𝑏

        # the difference b/w the prediction of the tested model (f_wb), 
        # and the actual value for the given example (y[i])
        diff = f_wb - y[i]  # 𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖)

        # loss is the cost for the given training example
        # because diff can be a negative number, it needs to be squared, 
        # and because it's squared, the loss increases rapidly when w is too small or too large
        loss = diff ** 2  # (𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))2

        # update the cost sum with the value of loss
        cost_sum = cost_sum + loss  # ∑𝑖=0𝑚−1(𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))2

    # calculate the total cost for the given model parameters - J(w, b)
    total_cost = (1 / 2 * m) * cost_sum  # 𝐽(𝑤,𝑏)=12𝑚∑𝑖=0𝑚−1(𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))2
    
    return total_cost



How to automate the process of finding the optimal w and b parameters?

In [8]:
def compute_gradient(x, y, w, b):
    """
    Computes the gradient for linear regression
    
    Args: 
        x (ndarray (m,)):       data, m examples
        y (ndarray, (m,)):      target values
        w, b (scalar):          model parameters
    Returns: 
        dj_dw (scalar):         the gradient of the cost w.r.t the parameters w
        dj_db (scalar):         the gradient of the cost w.r.t. the parameters b
    """
    m = x.shape[0]
    dj_dw = 0
    dj_db = 0

    for i in range(m): 
        f_wb = w * x[i] + b

        dj_dw_i = (f_wb - y[i]) * x[i]  # (𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))𝑥(𝑖)
        dj_db_i = f_wb - y[i]  # (𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))

        # update both variables simultaneously
        dj_dw += dj_db_i  
        dj_db += dj_dw_i 

    dj_dw = dj_dw / m  # 1/𝑚∑𝑖=0𝑚−1(𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))𝑥(𝑖)
    dj_db = dj_db / m  # 1/𝑚∑𝑖=0𝑚−1(𝑓𝑤,𝑏(𝑥(𝑖))−𝑦(𝑖))

    return dj_dw, dj_db


In [ ]:
def gradient_descent(x, y, w_initial, b_initial, alpha, number_of_iterations, cost_function, gradient_function): 
    """
    Performs gradient descent to fit w, b. 
    Updates w, b by taking number_of_iterations gradient steps with learning rate alpha

    Args: 
        x (ndarray (m,)):               data, m examples
        y (ndarray, (m,)):              target values
        w_initial, b_initial (scalar):  model parameters
        alpha (float):                  learning rate
        number_of_iterations (int):     number of iterations to run gradient descent
        cost_function:                  callback to produce cost
        gradient_function:              callback to produce gradient

    Returns: 
        w, b (scalar):                  updated values of parameters w, b after running gradient descent
        J_history (list):               history of cost values
        p_history (list):               history of parameters [w, b]
    """

    w = w_initial
    b = b_initial

    for i in range(number_of_iterations): 
        dj_dw, dj_db = gradient_function(x, y, w, b)

        w = w - alpha * dj_dw  # w = 𝑤−𝛼∂𝐽(𝑤,𝑏)∂𝑤
        b = b - alpha * dj_db  # b = 𝑏−𝛼∂𝐽(𝑤,𝑏)∂𝑏
    
    return w, b


Because the original data stored as json has much more features than we're going to need at the moment,
we need to reduce it: 
- from the list of features of each home, leave price, area and id
- filter out those homes, that have no price or no area
- parse the values to get integers/floats

In [9]:
import json

with open('../../output/pricePerArea-Sassari.json') as sassari_houses:
    file_contents = sassari_houses.read()
  
parsed_json = json.loads(file_contents)
print(parsed_json)

[[{'price': '€ 60.000', 'area': '168 m², esterna 974 m²\n                                         - Vedi dettaglio', 'id': 'EK-99220190 - 04/11/2022'}, {'price': '€ 64.000', 'area': '80 m²', 'id': '60622858 - 03/11/2022'}, {'price': '€ 64.000', 'area': '80 m²', 'id': '60621836 - 04/01/2023'}, {'price': '€ 65.000', 'area': '121 m² | commerciale 124 m²\n                                         - Vedi dettaglio', 'id': '18538758 - 01/09/2021'}, {'price': '€ 65.000', 'area': '250 m²', 'id': '40293092 - 28/11/2022'}, {'price': '€ 65.000', 'area': '30 m²', 'id': 'EK-99925384 - 12/12/2022'}, {'price': '€ 68.000', 'area': '105 m²\n                                         - Vedi dettaglio', 'id': '22111104 - 10/12/2022'}, {'price': '€ 69.000', 'area': '50 m²', 'id': '296504 - 11/01/2023'}, {'price': '€ 69.000', 'area': '32 m²', 'id': '23354189 - 24/09/2022'}, {'price': '€ 69.000', 'area': '50 m², esterna 15 m²\n                                         - Vedi dettaglio', 'id': 'EK-70753168 - 06/

Now that we have only the data we need in a json format, let's convert it into python array of floats: 

In [10]:
# Load our data set
x_training_examples = np.array([1.0, 2.0])   # stores features
y_training_examples = np.array([300.0, 500.0])   # stores targets
